<table align="left" style="border-style: hidden" class="table"> <tr> <td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon256.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Spring 2018</h4><p>Ani Adhikari</div></td></tr></table><!-- not in pdf -->

In [ ]:
# SETUP

import itertools
import numpy as np
from datascience import *
from prob140 import *

# These lines do some fancy plotting magicdef _remove_gaps(domain, prob):
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# These lines make warnings look nicer
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Useful for probability calculations
from scipy import stats
from scipy import misc

In [ ]:
def _remove_gaps(domain, prob):
    """
    domain must be sorted
    """
    new_probs = []
    index = 0
    max_value = max(domain)
    for i in range(max_value + 1):
        if i < domain[index]:
            new_probs.append(0)
        else:
            new_probs.append(prob[index])
            index += 1
    return np.array(new_probs)

In [ ]:
def plot_tails(dist, tick_distance=0.05):
    """
    Plots the tails of a discrete distribution. All values must be non-negative integers.
    
    Parameters
    ----------
    dist : Table
        2-column distribution.
    tick_distance : float (optional)
        Minimum distance between yticks (Default: 0.05).    
    """
    _domain, _prob = dist.sort(0).columns
    assert all(_domain == _domain.astype(int)), 'Values must be integer.'
    assert all(_domain >= 0), 'Tail sum only works for non-negative values'
    
    left_align = 0
    
    new_probs = _remove_gaps(_domain, _prob)
    max_value = _domain[-1]
    show_cdf = not(0 in new_probs)


    colors = list(itertools.islice(itertools.cycle(dist.chart_colors), int(max_value) + 1))
    cumprob = np.cumsum(new_probs)
    for i in range(1, max_value + 1):
        if i:
            plt.fill_between([left_align, i + left_align], cumprob[i - 1], cumprob[i], alpha=0.7, color=colors[i])
        else:
            plt.fill_between([left_align, i + left_align], cumprob[i], alpha=0.7, color=colors[i])            
    
    for p in cumprob:
        plt.plot([left_align, max_value + 1], [p, p], color='k', linewidth=2)
    plt.plot([left_align, max_value + 1], [0, 0], color='k', linewidth=2)
    plt.vlines(np.arange(max_value + 2), 0, 1, linewidth=2)
    plt.bar(np.arange(max_value + 1), cumprob, width=1, alpha=0.15, color='lightblue', align='edge')
    
    step = int(max_value / 15) + 1
    xticks = np.arange(0, max_value + 2, step)
    if show_cdf:
        space = max_value / 5
    
        for i in range(0, max_value + 1):
            plt.plot([-1 - space, 0 - space], [cumprob[i], cumprob[i]], color='k', linewidth=2)
            if i:
                plt.fill_between([-1 - space, 0 - space], cumprob[i - 1], cumprob[i], alpha=0.7, color=colors[i])
            else:
                plt.fill_between([-1 - space, 0 - space], cumprob[i], alpha=0.7, color=colors[i])
        plt.plot([-1 - space, -space, -space, -1 - space, -1 - space], [0, 0, 1, 1, 0], color='k', linewidth=2)
        plt.xlim(-1.55 - space, max_value + 1.05)
        plt.xticks(np.append(-space - 0.5, xticks), np.append('Dist', xticks))
    else:
        plt.xticks(xticks, xticks)
    
    ax = plt.gca()
    ax.grid(False)
    plt.ylim(-.05, 1.05)
    
    yticks = [1]
    for p in cumprob[::-1]:
        if p <= yticks[-1] - tick_distance:
            yticks.append(p)
    plt.yticks(yticks)
    ax.spines['left'].set_position(('data', -0.05))

    plt.title('CDF and Tails')

In [ ]:
def plot_sample(p_array, s, m):
    """
    Plots a dot plot sampled s times from p_array.
    
    Parameters
    ----------
    p_array : array
        An array of probabilities corresponding to p_0, ..., p_n.
    s : int
        Sample size.
    m : int
        Highlighted value.
    """
    assert s % 2 == 1, 's must be odd.'
    n = len(p_array)
    samples = sorted(np.random.choice(np.array(n), size=s, p=p_array))
    counts = _remove_gaps(*np.unique(samples, return_counts=True))
    for i in range(n):
        if i < len(counts):
            x = [i] * counts[i]
            y = np.arange(counts[i])
            if i == m:
                color = 'k'
                plt.vlines(i, -1, max(10, max(counts)), linewidth=1, color='blue', lw=2, label='m')
            else:
                color = 'k'
            plt.scatter(x, y, color=color, s=40)
    median_value = samples[s // 2]
    plt.axhline(-1, color='k')
    plt.xlim(-0.5, n - 0.5)
    ax = plt.gca()
    ax.set_xticks(np.arange(n))
    ax.set_yticks([])
    ax.grid(False)
    plt.ylim(-1, max(10, max(counts)))
    plt.legend()
    if median_value > m:
        print('Sample median is greater than m.')
    else:
        print('Sample median is not greater than m.')
    plt.title('Sample Median')


# Lab 4: Expectations of Discrete Order Statistics #
Additivity is a powerful property of expectation: you have seen several examples in which writing a random variable as a sum of simpler variables has led to a simple calculation of its expected value. But other methods are needed for finding the expectations of random variables that can't easily be written as sums. 

Using the basic definition of expectation is one such method, and it works well when you can easily write down the distribution of the random variable and have the computational tools to work out the resulting sum. However, distributions aren't always strightforward to write down.

This lab is about expectations of a class of random variables that don't have natural representations as sums of simpler variables. These variables, called *order statistics*, can be thought of as the elements of a random sample arranged in increasing order. We will work with just two of them:
- the minimum
- the median

You have used the sample median as a statistic in Data 8. How big is a random sample median expected to be? By the end of this lab you will have an answer to this question. 

The corresponding question for the sample mean has an easy answer by additivity: the expectation of a random sample mean is the population mean. But additivity isn't very helpful for medians as medians aren't easy to think of as sums. A different approach is needed. 

What you will learn in this lab:
- What tail probabilities are, and what they have to do with expectation
- How to use tail probabilities to find expectations of the minimum and median values of random samples
- What the distribution of the sample median looks like when the sample is large


Let's start out by noticing a clever way of finding the expectation of a non-negative integer valued random variable.

## Part 1: Tail Probabilities ##
Let $X$ be a random variable with values $0, 1, 2, 3, \ldots, N$ for some fixed integer $N$. As you know, the cumulative distribution function (cdf) of $X$ is the function $F$ defined by
$$
F(x) ~ = ~ P(X \le x) 
$$
for all $x$.

Define the *tail probability function* $T$ by

$$
T(x) ~ = ~ P(X > x) ~ = ~ 1 - F(x)
$$
for all $x$.

Here is the probability histogram of a random variable with distribution `example_dist`. The gold area is equal to $F(4)$ and the blue area is $T(4)$.

In [ ]:
example_vals = np.arange(8)
example_probs = make_array(0.05, 0.1, 0.2, 0.25, 0.15, 0.12, 0.08, 0.05)

example_dist = Table().values(example_vals).probability(example_probs)

Plot(example_dist, event = np.arange(5))

plt.title('Gold Area = $F(4)$, Blue Area = $T(4)$');

The goal of this part of the lab is for you to discover a relation between the tail probabilities and the expectation of $X$, when $X$ has a finite number of non-negative integer values.

For simplicty of notation, define $p_i = P(X = i)$ for $0 \le i \le N$. Then $p_i \ge 0$ for all $i$, $\sum_{i=0}^N p_i = 1$, and

$$
E(X) ~ = ~ p_1 + 2p_2 + 3p_3 + \cdots + Np_N
$$

### 1a) Computing Tail Probabilities ###
For a random variable $X$ with possible values $0, 1, 2, \ldots, N$ as above, we will use the term "probability array" for an array consisting of the sequence $p_0, p_1, \ldots, p_N$. 

Define a function `tails` that takes the probability array as its argument and returns an array of all the tail probabilities.

In [ ]:

def tails(prob_array):
    return ...

Suppose $p_0 = 0.3$, $p_1 = 0.5$, and $p_2 = 0.2$, and let `probs` be the array [0.3, 0.5, 0.2].

What should `tails(probs).item(0)` and `tails(probs).item(2)` evaluate to? Check that they work out as they should.

In [ ]:
probs = make_array(0.3, 0.5, 0.2)

tails(probs).item(0), tails(probs).item(2)

### 1b) Expectation and the Tails ###
Here is the distribution of a random variable $X$.

In [ ]:
values_X = np.arange(5)
probs_X = make_array(0.3, 0.35, 0.2, 0.1, 0.05)
dist_X = Table().values(values_X).probability(probs_X)
dist_X

The `prob140` method `ev` operates on a Distribution object and returns the expectation. Run the cell below to get $E(X)$.

In [ ]:
dist_X.ev()

Run the cell below. The colorful bar on the left hand side of the figure shows the probabilities $p_0, p_1, p_2, p_3$, and $p_4$ as areas of rectangles stacked vertically, with $p_0$ on the bottom, $p_1$ above that, $p_2$ above $p_1$, and so on.

The figure on the right is a rectangle of height 1. As its title says, the figure displays the cdf and tail probabilities of $X$. Identify which part is the cdf and which are the tail probabilities. That is, for each integer $x$ in the range 0 through 4, locate the area representing $F(x)$ and also the corresponding $T(x)$. You don't have to write out an answer, but you might find some scratch paper helpful.

In [ ]:
plot_tails(dist_X)

Now look at the regions showing the tail probabilities $T(0)$ through $T(4)$, and look again at the formula for $E(X)$ given earlier in this part of the lab. How can you get $E(X)$ by using the tails probabilities, with no subtraction or multiplication?

Once you have figured it out, complete the cell below so that the line of code evaluates to the expectation that you computed in the previous cell.

In [ ]:

...(tails(probs_X))

### 1c) A New Formula for Expectation ###
For a random variable $X$ with values $0, 1, 2, \ldots, N$ and probability array $[p_0, p_1, \ldots, p_N]$, define a function `ev_by_tails` that takes the probability array as its argument and returns $E(X)$ using the method you discovered in part **b**. Use the function `tails` you defined in part **a**.

In [ ]:

def ev_by_tails(prob_array):
    return ...

If the distribution of $X$ is uniform on $0, 1, 2, 3, 4$, then what should $E(X)$ be? Run the cell below to make sure your function does the right thing.

In [ ]:
ev_by_tails((1/5) * np.ones(5))

### 1d) When Some $p_i$'s are Zero ###

Stacking the probabilities vertically is problematic when some of the $p_i$'s are 0, because you can't see where the 0's are in the stack. But the cdf (and hence the tails) can be plotted just fine.

Here is an example of a distribution in which $p_0 = 0$. Notice that the yellow bar shows $p_1$, and confirm that you understand the rest of the figure as well. You don't have to write anything.

Also notice that since the possible values of $Y$ are $1, 2, 3$, the expectation $E(Y)$ has to be greater than 1. This is apparent from the figure, but not so apparent from the basic formula for expectation without a little calculation.

In [ ]:
values_Y = np.arange(4)
probs_Y = make_array(0, 0.25, 0.5, 0.25)
dist_Y = Table().values(values_Y).probability(probs_Y)

plot_tails(dist_Y)

Run the cell below to confirm that that your function `ev_by_tails` does the right thing in this case too.

In [ ]:
dist_Y.ev(), ev_by_tails(probs_Y)

Here is another example; this time $p_2 = 0$. 

In [ ]:
values_Z = np.arange(4)
probs_Z = make_array(0.25, 0.5, 0, 0.25)
dist_Z = Table().values(values_Z).probability(probs_Z)

plot_tails(dist_Z)

Run the cell below to confirm that the calculation of the expectation works out by both methods.

In [ ]:
dist_Z.ev(), ev_by_tails(probs_Z)

What you have discovered visually is called the *tail sum formula* for expectation. The top of page 172 of the text by Pitman has an algebraic proof that mirrors your observations.

The tail sum formula provides a simple way of finding the expectation of a non-negative integer valued random variable, by adding the tail probabilities. 

The rest of the lab consists of applications of this method to find expectations of random variables for which the tail probabilities are easier to calculate than the distribution.

#newpage

## Part 2: Sample Minimum ##
Let $X_1, X_2, \ldots, X_n$ be i.i.d., each with possible values $0, 1, 2 \ldots N$. Let $S$ be the smallest of the $n$ sampled values, that is,

$$
S ~ = ~ \min\{X_1, X_2, \ldots , X_n\}
$$

Let $T_S$ be the tail probability function of $S$, that is, for all $x$ let $T_S(x) = P(S > x)$. 

Let $T_X$ be the tail probability function of $X_1$. Earlier in the course you have seen that there is a simple relation between $T_S$, $T_X$, and $n$. Get a piece of scratch paper and remind yourself of that relation.

### 2a) Expectation of the Sample Minimum ###
Define a function `expected_sample_min` that takes the probability array of $X_1$ as the first argument and $n$ as the second, and returns $E(S)$. Use the function `tails` that you defined in Part 1.

In [ ]:

def expected_sample_min(prob_array, n):
    return ...

The expectation of the minimum of two rolls of a die is about 2.53, as you should check by hand when you get a chance. Run the cell below to confirm that your function `exepected_min` agrees, but first figure out why there's a 0 in the probability array.

In [ ]:
expected_sample_min(np.append(0, (1/6)*np.ones(6)), 2)

### 2b) Sample Minimum Household Size ###
The website Statista provides the [distributions of household sizes](https://www.statista.com/statistics/242189/disitribution-of-households-in-the-us-by-household-size/) in the United States, for various years. Go to the website and notice their choice of stacked bars representing the proportions of households of different sizes. Notice also:
- No household has 0 members, so the bottom bar (blue) represents households with one member.
- The distributions have been *truncated* at 7. That is, the last category of sizes is "7 or more". This truncation could markedly affect the calculation of average household size, so we won't do that. However, it will have little effect on the minimum household size in the sample and on the sample median. 

Let's start with the 2016 distribution. We'll call the final category "7 persons", not "7 or more". So everything we do using these proportions will be an approximation, not an exact value.

First, run the cell below for some cleanup to compensate for the fact that the proportions provided don't quite add up to 1. The array `hh_size_2016` is the household size probability array for 2016: there are no households with 0 members, roughly 28% with 1 member, and so on. We have just normalized Statista's proportions by the total and rounded the results.

In [ ]:
statista = make_array(0.2813, 0.3401, 0.1544, 0.1293, 0.06, 0.0224, 0.0127)
hh_size_2016 = np.append(0, statista)
hh_size_2016 = np.round(hh_size_2016/sum(hh_size_2016), 6)
hh_size_2016

Find the expected minimum household size in a random sample of 100 households.

In [ ]:

expected_sample_min(...)

Does the answer agree with your intuition? Explain


**Your answer here.**

Now do the same using the 1970 distribution. We have set up the 1970 household size probability array.

In [ ]:
hh_size_1970 = np.append(0, make_array(.17, .29, .17, .16, .1, .06, .05))

In [ ]:

...

#newpage

## Part 3: Sample Median ##
We will analyze the sample median in the straightforward case where the sample size $2n+1$ is odd and "the median" is defined as the $(n+1)$st value when the sample is sorted in ascending order. The $(n+1)$st is the right item to pick out, because $2n+1 = n + 1 + n$.

This sample median is formally known as the *$(n+1)$st order statistic* of the sample.

The example below will help clarify the definition.

In [ ]:
# 7 = 2*3 + 1

sample_7 = make_array(23, 81, 34, 13, 56, 27, 26)
sample_7_median = np.sort(sample_7).item(3)       # item(3) is the 4th item
sample_7_median

### 3a) The Event "Sample Median $ > m$" ###
Start by visualizing what has to happen for the sample median to be larger than a given value. The function `plot_sample` takes three arguments:
- a probability array of our usual form $[p_0, p_1, \ldots, p_N]$
- an odd numbered sample size $s$
- a value of $m$

It displays a dot plot of an i.i.d. sample of size $s$ drawn from the probability array (remember that the possible values are $0, 1, \ldots, N$). The vertical blue line is at $m$. The statement above the plot says whether or not the sample median is greater than $m$.

Run the cell below several times just as written. Each time, count how many dots are on the right of the vertical line, and note whether or not the median is bigger than $m$. Then change $m$ and $s$ and run it again a few times, but don't go too crazy with the changes.

In [ ]:
plot_sample(hh_size_1970, 7, 2)

Fill in the blank with the appropriate phrase:

The median of a sample of size $2n+1$ is bigger than $m$ if and only if $\underline{~~~~~~~~~~~~~~~~}$ of the sampled elements are bigger than $m$.


**Your answer here.**

### 3b) Tails of the Sample Median ###
Let $X_1, X_2, \ldots , X_{2n+1}$ be i.i.d., each with possible values $0, 1, 2, \ldots, N$, c.d.f $F_X$, and tail probability function $T_X$. 

Let $M$ be the median of $X_1, X_2, \ldots, X_{2n+1}$, as defined above. Let $T_M$ be the tail probability function of $M$.

Fill in the first blank below with the name of a famous distribution and the appropriate parameters. Fill in the second blank with an integer. **Explain your choices.**

For an integer $m$ in the range 0 through $N$, $T_M(m)$ is equal to the probability that a random variable with $\underline{~~~~~~~~~~~~~~~~~~~~~}$ distribution is greater than $\underline{~~~~~~~~~~~~}$.

[Hint: Use part **a**. Define the $X$'s as trials and figure out how to define "success".]

Typographical note: You can type T_M for $T_M$ and T_X for $T_X$.


** Your answer here.**


### 3c) Expectation of the Sample Median ###
Define a function `expected_sample_median` that takes as its arguments a probability array of the form $[p_0, p_1, \ldots, p_N]$ and a sample size, and returns:
- the string `'For this calculation, the sample size has to be an odd number.'` if the sample size is even
- the expectation of the median of an i.i.d. sample of the given size from the given array, if the sample size is odd

You're going to need the function `tails` you defined in **1a**, and an expression for $n$ in terms of $s = 2n+1$. 

In [ ]:

def expected_sample_median(prob_array, s):
    if s % 2 > 0:
        return ...
    else:
        return ...

Run the cell below to confirm that your function is doing the right thing. It uses the probability array `probs_X` from **1b**.

In [ ]:
expected_sample_median(probs_X, 10)

If the sample size is 1, then there's just one element $X_1$ in the sample, and that's the median. So if the sample size is 1 then $M = X_1$ and hence $E(M) = E(X_1)$. 

The cell below uses the probability array `probs_X` from **1b**. Check that your function does the right thing in the cell below; the value you should get is in **1b**.

In [ ]:
expected_sample_median(probs_X, 1)

### 3d) Sample Median Household Size ###

Find the expectation of the median household size in a random sample of 25 households taken in 2016.

In [ ]:

...

Do the same for a random sample of 25 households in 1970.

In [ ]:

...

#newpage

## Part 4: Median of a Large Sample ##
Now take a look at what happens to the sample median when the sample size increases.

### 4a) Guessing the Sample Median ###
If someone took a large random sample from the population of U.S. households in 2016, roughly what would you expect the median household size in the sample to be? Why?

Remember that we are using the array `hh_size_2016` as the distribution of household sizes.

In [ ]:
hh_size_2016


**Your answer here.**

Answer the same question (you don't have to write out your answer) for the 1970 distribution, and then look at the vertical scale on the [visualization in the source](https://www.statista.com/statistics/242189/disitribution-of-households-in-the-us-by-household-size/) of the data. Make sure your answers to this exercise are consistent with what you see there.

### 4b) Plotting the Expected Sample Median ###
Define a function `exp_median_2016` that takes an odd sample size $s$ as its argument and returns the expected median size of an i.i.d. sample of $s$ housedholds from the distribution specified by `hh_size_2016`. Just re-use the relevant code from **3c**.

In [ ]:

def exp_median_2016(s):
    return ...

Complete the cell below to generate a plot of the expected sample median as a function of sample size. 

In [ ]:

exp_median_tbl = Table().with_column('Sample Size', np.arange(1, 1000, 2))
exp_medians = exp_median_tbl...
exp_median_tbl = exp_median_tbl.with_column('Expected Sample Median, 2016', exp_medians)
exp_median_tbl.scatter(0)

Is the graph consistent with your answer to part **a**?


**Your answer here.**

Now include the corresponding data from 1970 as well.

In [ ]:

def exp_median_1970(s):
    return ...

exp_medians = exp_median_tbl...
exp_median_tbl = exp_median_tbl.with_column('Expected Sample Median, 1970', exp_medians)
exp_median_tbl.scatter(0)

### 4c) Distribution of the Median of a Large Sample ###
In **3d** you found the expected median of a sample of size 25 from the 2016 distribution.

Complete the cell below to display the histogram of an empirical distribution of 20,000 simulated values of the median of an i.i.d. random sample of size 25 taken from the household size distribution of 2016. 

Helpful methods:
- The `prob140` Table method [`sample_from_dist`](https://probability.gitlab.io/prob140/_autosummary/prob140.Table.sample_from_dist.html#prob140.Table.sample_from_dist)
- np.median, which computes the median of an odd-numbered set exactly as we are doing here
- The `prob_140` method [`emp_dist`](https://probability.gitlab.io/prob140/tutorial.html#empirical-distributions)

In [ ]:

# Simulated median of n iid draws

# A Distribution object
hh_2016_dist = Table().values(range(8)).probability(hh_size_2016) 

n = 25
repetitions = 20000

medians = ...
for i in range(...):
    sample = hh_2016_dist...
    medians = np.append(medians, np.median(sample))

print('IID Sample Size ', str(n))
print('Average of', str(repetitions), 'Sample Medians =', str(np.mean(medians)))

...

First check that the empirical average of all the simulated medians is close to the expected value you got in **3d**.

Next note that even with a sample size of just 25, the sample median is overwhelmingly likely to be what you think it should be. You're welcome to increase the sample size (make sure it's still odd, for comparability) and re-run the cell to see what happens.

Note the **difference between sample means and sample medians when drawing from a discrete distribution**: As the sample size increases, the distribution of the sample mean becomes roughly normal by the Central Limit Theorem as you saw in Data 8, but the distribution of the sample median puts almost all its mass on one or two points.

The cdf of the underlying distribution of 1970 households is pretty close to 0.5 at 2, though it actually crosses 0.5 at 3. This leads to the distribution of the sample median being less clustered than its 2016 counterpart when the sample size is small to moderate.

Copy-paste the code from the cell above, keep n at 25, change 2016 to 1970, and see what happens.

In [ ]:

...

## Conclusion ##
What you have learned in this lab:
- An alternative approach to finding expectations of some random variables that don't have good representations as sums
- The *tail sum formula* for the expectation of a non-negative integer valued random variable
- The use of the tail sum formula in finding the expectation of the minimum of an i.i.d. sample from a population of non-negative integers
- A duality between the tail of the median of an i.i.d. sample and the tail of a well known distribution
- The use of this duality and the tail sum formula in finding the expectation of the median of an i.i.d. sample
- When the sample is large, the distribution of the sample median is very likely to be concentrated at one or two values around where the cdf of the underlying population crosses 0.5

## Submission Instructions

1. **Save your notebook using File > Save and Checkpoint.**
2. Run the cell below to generate a pdf file.
3. Download the pdf file and confirm that none of your work is missing or cut off.
4. Submit the assignment to Lab_04b on Gradescope.

#### Logistics 

1. Examine the generated pdf before uploading to make sure that it contains all of your work.
2. When submitting to Gradescope, select the pages of your upload corresponding to each question. 
3. If you encounter any difficulties when submitting or exporting your assignment, please make a private Piazza post **before the deadline**. 

### **We will not grade assignments which do not have pages selected for each question or were submitted after the deadline.** 

In [ ]:
import gsExport
gsExport.generateSubmission("Lab_04.ipynb")